In [5]:
import torch

# For reproducibility
torch.manual_seed(42)

# Let's say we have 1000 employees, each with 5 features:
# [age, tenure_years, engagement_score, manager_rating, salary_normalized]
n_samples = 1000
n_features = 5

X = torch.randn(n_samples, n_features)

# Create a fake "true" rule for attrition:
# higher engagement & manager_rating -> lower attrition
weights_true = torch.tensor([-0.2, -0.1, -1.0, -0.8, 0.1])  # arbitrary
logits = X @ weights_true + 0.2 * torch.randn(n_samples)  # add noise

# Convert logits to probabilities with a sigmoid
probs = torch.sigmoid(logits)

# Sample 0/1 attrition from these probabilities
y = torch.bernoulli(probs).long()

X.shape, y.shape

(torch.Size([1000, 5]), torch.Size([1000]))

In [9]:
# Create train/test split (80% train, 20% test)
n_samples = X.shape[0]
indices = torch.randperm(n_samples)

train_size = int(0.8 * n_samples)
train_idx = indices[:train_size]
test_idx = indices[train_size:]

X_train = X[train_idx]
y_train = y[train_idx]

X_test = X[test_idx]
y_test = y[test_idx]

X_train.shape, X_test.shape


(torch.Size([800, 5]), torch.Size([200, 5]))

In [6]:
import torch.nn as nn

class AttritionNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = AttritionNet(input_dim=n_features)
model


AttritionNet(
  (fc1): Linear(in_features=5, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)

In [10]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

X_train_float = X_train.float()
y_train_float = y_train.float()

n_epochs = 50

for epoch in range(1, n_epochs + 1):
    model.train()
    
    optimizer.zero_grad()
    preds = model(X_train_float).squeeze()
    loss = criterion(preds, y_train_float)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:02d} - Train Loss: {loss.item():.4f}")


Epoch 10 - Train Loss: 0.5281
Epoch 20 - Train Loss: 0.5241
Epoch 30 - Train Loss: 0.5206
Epoch 40 - Train Loss: 0.5175
Epoch 50 - Train Loss: 0.5140


In [11]:
model.eval()
with torch.no_grad():
    X_test_float = X_test.float()
    probs_test = model(X_test_float).squeeze()
    
    # Convert probabilities to class predictions (0 or 1) with threshold 0.5
    y_pred = (probs_test >= 0.5).long()
    
    accuracy = (y_pred == y_test).float().mean().item()

print(f"Test accuracy: {accuracy:.3f}")


Test accuracy: 0.670


In [12]:
# Example "employee": [age, tenure, engagement, manager_rating, salary_norm]
example = torch.tensor([[35.0, 3.0, 4.2, 3.8, 0.5]])
with torch.no_grad():
    risk = model(example.float()).item()

print(f"Predicted attrition risk: {risk:.3f}")


Predicted attrition risk: 0.000
